Objectives:
Step 4 of the wildfire-lake-productivity project

1. Define functions to use Google Earth Engine (GEE or ee) data
        a. cloud mask
        b. date/ time indicators
        c. vegetation indices
        d. transfer ee feature properties to a dictionary

2. Import burned catchment dataset 
 
3. Select relevant Landsat images (date and area of interest)
   
4. Generate table of vegetation index (VI) data

5. Export table to Google Drive

Run with wildfire-lake-productivity environment

# 1.  Import packages

In [52]:
import ee
import os
from io import BytesIO
import earthpy as et

import geemap
import folium
import geopandas as gpd
import pandas as pd
import time
import requests



try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

# 2. Set working directory

In [2]:
# Set working directory

# if the desired path exists:
data_dir = os.path.join(et.io.HOME, 'Dropbox',
                        'cu_earthdata_certificate_2021', 'earthlab_project', 'data')
if os.path.exists(data_dir):
    # set working directory:
    os.chdir(data_dir)
else:
    print("Be sure you have run first notebook to generate directory and required data")

In [114]:
# def l8cloudmask(img):
#     fillBitMask = 1
#     clearBitMask = 1 << 1
#     waterBitMask = 1 << 2
#     cloudShadowBitMask = 1 << 3
#     snowBitMask = 1 << 4
#     cloudConfBitMask = 1 << 7
# #     cirrusConfBitMask = 1 << 9
#     occlusionBitMask = 1 << 10

#     ls_bands = ee.List(['B4', 'B5', 'B6'])
#     pixelqa_name = 'BQA'

#   # Get the pixel QA band
#     pixelqa = img.select(pixelqa_name)

#     mask = pixelqa.bitwiseAnd(fillBitMask).eq(0).And(pixelqa.bitwiseAnd(clearBitMask).neq(0)).And(pixelqa.bitwiseAnd(waterBitMask).eq(0)).And(pixelqa.bitwiseAnd(cloudShadowBitMask).eq(0)).And(pixelqa.bitwiseAnd(snowBitMask).eq(0)).And(pixelqa.bitwiseAnd(cloudConfBitMask).eq(0)).And(pixelqa.bitwiseAnd(cirrusConfBitMask).eq(0)).And(pixelqa.bitwiseAnd(occlusionBitMask).eq(0))  
# # Return the masked image (excluding the PIXELQA and STQA layers)
#     return img.updateMask(mask).select(ls_bands)

In [211]:
# Import file containing full catchment for each burned lake
# from local drive generated in 01_download_clip_merge.ipynb
file_path = os.path.join('land_only_whole_ctchmt_aqst.shp')

# Geodataframe used in folium map
land_catchmts = gpd.read_file(file_path)

# EE object used for Earth Engine collections and calculations
#small_gdf = land_catchmts[["Hylak_id", "geometry"]]
land_ctchmts_ee = geemap.geopandas_to_ee(land_catchmts)

In [212]:
Map = geemap.Map()
Map.addLayer(land_ctchmts_ee, {}, "geopandas to ee")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [118]:
id_list = land_catchmts['landsat_id'].tolist()

In [205]:
reduce_vi = create_reduce_regions_function(
    collection=land_ctchmts_ee, reducer=ee.Reducer.mean(), scale=30, crs='EPSG:4326')

collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
    .filter(ee.Filter.inList('LANDSAT_ID', ee.List(id_list)))\
    .map(add_ndvi)\
    .select('NDVI')
# .select(['B4', 'B5', 'B6', 'B7', 'pixel_qa'])\
# .map(add_nbr)\
# .map(cfmask)\
# .map(add_ndvi)

median = collection.median()
# def reduce_salt(image):
#   reduced = image.reduceRegions(
#               reducer=ee.Reducer.mean(),
#               collection=land_ctchmts_ee,
#               scale=30)
#   return ee.Feature(None, reduced) 

                                    
# pdsi_stat_fc = ee.FeatureCollection(collection.map(reduce_salt))#.filter(
#     #ee.Filter.notNull(collection.first().bandNames()))

# pdsi_stat_fc=fc_to_dict(pdsi_stat_fc).getInfo()

# print(type(pdsi_stat_fc), '\n')
# for prop in featureCollection.keys():
#     print(prop + ':', featureCollection[prop][0:3] + ['...'])

In [206]:
js_snippet = """
// Load input imagery: Landsat 7 5-year composite.
var image = ee.Image('LANDSAT/LE7_TOA_5YEAR/2008_2012');

// Load a FeatureCollection of counties in Maine.
var maineCounties = ee.FeatureCollection('TIGER/2016/Counties')
  .filter(ee.Filter.eq('STATEFP', '23'));

// Add reducer output to the Features in the collection.
var maineMeansFeatures = image.reduceRegions({
  collection: maineCounties,
  reducer: ee.Reducer.mean(),
  scale: 30,
});

// Print the first feature, to illustrate the result.
print(ee.Feature(maineMeansFeatures.first()).select(image.bandNames()));
"""

In [207]:
geemap.js_snippet_to_py(js_snippet, add_new_cell=True, import_ee=True, import_geemap=True, show_map=True)

In [210]:
import ee
import geemap

Map = geemap.Map()

# Load input imagery: Landsat 7 5-year composite.
image = ee.Image('LANDSAT/LE7_TOA_5YEAR/2008_2012')

# Load a FeatureCollection of counties in Maine.
maineCounties = ee.FeatureCollection('TIGER/2016/Counties') \
  .filter(ee.Filter.eq('STATEFP', '23'))

# Add reducer output to the Features in the collection.
maineMeansFeatures = image.reduceRegions({
  'collection': maineCounties,
  'reducer': ee.Reducer.mean(),
  'scale': 30
})

# Print the first feature, to illustrate the result.
#print(ee.Feature(maineMeansFeatures.first()).select(image.bandNames()))
Map


EEException: Unrecognized argument type to convert to a FeatureCollection: {'collection': <ee.featurecollection.FeatureCollection object at 0x7fb252204910>, 'reducer': <ee.Reducer object at 0x7fb252204d50>, 'scale': 30}

In [202]:
# def getmeans(image):
#      return ee.Feature(image.reduceRegions(collection=land_ctchmts_ee, reducer=ee.Reducer.mean(), scale=30))


# # Map the function over the ImageCollection.
fc = ee.FeatureCollection(collection.map(reduce_salt))

func=fc_to_dict(fc).getInfo()

# print(type(fc), '\n')
# for prop in featureCollection.keys():
#     print(prop + ':', featureCollection[prop][0:3] + ['...'])


EEException: Collection.first: Error in map(ID=LC08_032034_20150504):
Feature, argument 'metadata': Invalid type.
Expected type: Dictionary<Object>.
Actual type: FeatureCollection.

In [ ]:
# print out the number of images in the ImageCollection
count = collection.size().getInfo()
print("Count: ", count)

In [ ]:
pdsi_dict = fc_to_dict(pdsi_stat_fc).getInfo()

In [176]:
print(type(pdsi_dict), '\n')
for prop in pdsi_dict.keys():
    print(prop + ':', pdsi_dict[prop][0:3] + ['...'])

<class 'dict'> 

millis: [1430760664760, 1445967120780, 1462555890660, '...']
system:index: ['LC08_032034_20150504', 'LC08_032034_20151027', 'LC08_032034_20160506', '...']


In [83]:
# select three Landsat 8 imagery in the Southwest area with the least cloud coverage
# collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
#     .filterDate('2017-03-01', '2017-04-30')\
#     .select(['B4', 'B5', 'B6', 'B7', 'pixel_qa'])\
#     .filterBounds(land_ctchmts_ee)

collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
          .select(['B4', 'B5', 'B6', 'B7', 'pixel_qa'])\
          .filter(ee.Filter.inList('LANDSAT_ID',ee.List(id_list)))
          .map(add_nbr)
          .map(add_ndvi)

# # calculate NDVI for each image using the map fucntion
# #vi_images = collection.map(addTime)
# vi_images = collection.map(cfmask)

# # calculate VI for each image using the map fucntion
# vi_images = vi_images.map(add_ndvi)
# vi_images = vi_images.map(add_nbr)
# #vi_images = vi_images.map(set_year)


# def eeAlgo(image):
#     reduced = image.reduceRegions(collection=land_ctchmts_ee, reducer=ee.Reducer.mean(), crs='EPSG:4326', scale=30)
#     image = image.set('value', reduced)
#     return image


def getmeans(image):
    return ee.Feature(image.reduceRegions(collection=land_ctchmts_ee, reducer=ee.Reducer.mean(), scale=30))

# Map the function over the ImageCollection.
featureCollection = ee.FeatureCollection(collection.map(getmeans))



#vi_median = vi_images.median()


#ls8_reduced_mean = ee.FeatureCollection(img.reduceRegions(reducer=ee.Reducer.mean(),
#                                        collection=land_ctchmts_ee,
#                                        scale=30))


# def nestedMappedReducer(featCol, imgCol):
#     def mapReducerOverImgCol(feat):
#         def imgReducer(img):
#             return img.reduceRegions(
#                 collection = ee.FeatureCollection(feat),
#                 reducer = ee.Reducer.first(),
#                 scale = 30
#             )

#         return imgCol.map(imgReducer).flatten()

#     return featCol.map(mapReducerOverImgCol).flatten()


# tblmaybe = nestedMappedReducer(land_ctchmts_ee,vi_images)



# def reduce_imgs(img):
#     reducedimg = img.reduceRegions(**{
#       'collection': land_ctchmts_ee,
#       'reducer': ee.Reducer.mean(),
#       'scale': 30
# })
#     return reducedimg

# reduced_vis = vi_images.map(reduce_imgs).flatten()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [96]:
pdsi_stat_fc

In [203]:
task = ee.batch.Export.table.toDrive(**{
    'collection': fc,
    'description': 'vi_mock_export',
    'folder':'new_example_folder',
    'fileFormat': 'CSV'
})

task.start()

In [204]:
while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(15)

Polling for task (id: OP762YRMFFAIBTWALBWUR5WA).
Polling for task (id: OP762YRMFFAIBTWALBWUR5WA).


In [141]:
taskParams = {
    'driveFolder': 'image',
    'fileFormat': 'CSV'   # CSV, KMZ, GeoJSON
}

# export all features in a FeatureCollection as one file
task = ee.batch.Export.table(pdsi_stat_fc, 'export_fc', taskParams)
task.start()

In [183]:
Map = emap.Map(center=[37,-106], zoom=6)
Map.add_basemap('ROADMAP') # Add Google Map
Map


vizParams = {
     'bands': ['NDVI'],
     'min': 0, 
     'max': 1
 }

#Map.centerObject(land_ctchmts_ee, 8)

Map.addLayer(collection,vizParams,'basic_images')

#Map.addLayer(vi_images,{},'vi_trend')

#Map.addLayer(vi_median, vizParams, 'Landsat image ')

#Map.addLayer(land_ctchmts_ee, {}, 'burnedlakes')
    
Map.addLayerControl()
Map

Map(center=[37, -106], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [ ]:

    
r = requests.get('https://drive.google.com/drive/u/0/folders/1gBj3y6etTKwX3Dm6JT3znLj7OmxXsBCV&output=csv')
data = r.content
    
df = pd.read_csv(BytesIO(data), index_col=0,parse_dates=['system:time_start'])


In [148]:
# Define a function to calculate NBR.
def add_nbr(img):
    """calculates the normalized burn ratio (nbr) using imported Landsat 8 bands
    Parameters
    -----------
    image: Landsat image(s) imported from earth engine
    Returns
    -----------
    band : image with added nbr band
    """
    nbr = img.normalizedDifference(
        ['B5', 'B7']).select([0], ['NBR'])
    return img.addBands(nbr)

# Define a function to calculate NDVI.
def add_ndvi(img):
    """calculates the normalized difference vegetation index (ndvi) using imported Landsat bands
    Parameters
    -----------
    image: Landsat image(s) imported from earth engine
    Returns
    -----------
    band : image with added ndvi band
    """
    ndvi = img.normalizedDifference(
        ['B5', 'B4']).select([0], ['NDVI'])
    return img.addBands(ndvi)

In [4]:
# This function adds a band representing the image timestamp.
def addTime(image):
    """selects image date from image metadata and adds to image as a band
    Parameters
    -----------
    image: Landsat image(s) imported from earth engine
    Returns
    -----------
    band : image with added time band
    """
    return image.addBands(image.metadata('LANDSAT_ID'))

# Define a function to add year as an image property.


def set_year(img):
    """selects image year from image metadata and adds to image as a band
    Parameters
    -----------
    image: Landsat image(s) imported from earth engine
    Returns
    -----------
    band : image with added year band
    """
    year = ee.Image(img).date().get('year')
    return img.addBands(year)

In [5]:
# Define a function to mask out clouds and cloud shadows.
def cfmask(img):
    """masks clouds and cloud shadows based on values in Landsat's pixel_qa band 
    Parameters
    -----------
    image: Landsat image(s) imported from earth engine
    Returns
    -----------
    band : image with clouds masked
    """
    cloud_shadow_bi_mask = 1 << 3
    cloud_bit_mask = 1 << 5
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloud_shadow_bi_mask).eq(0).And(
        qa.bitwiseAnd(cloud_bit_mask).eq(0))
    return img.updateMask(mask)

In [113]:
def create_reduce_regions_function(collection,
                                  reducer=ee.Reducer.mean(),
                                  scale=30,
                                  crs='EPSG:4326'):
  """Creates a regions reduction function.

  Creates a region reduction function intended to be used as the input function
  to ee.ImageCollection.map() for reducing pixels intersecting a provided region
  to a statistic for each image in a collection. See ee.Image.reduceRegions()
  documentation for more details.

  Args:
    geometry:
      An ee.Geometry that defines the region over which to reduce data.
    reducer:
      Optional; An ee.Reducer that defines the reduction method.
    scale:
      Optional; A number that defines the nominal scale in meters of the
      projection to work in.
    crs:
      Optional; An ee.Projection or EPSG string ('EPSG:5070') that defines
      the projection to work in.
    bestEffort:
      Optional; A Boolean indicator for whether to use a larger scale if the
      geometry contains too many pixels at the given scale for the operation
      to succeed.
    maxPixels:
      Optional; A number specifying the maximum number of pixels to reduce.
    tileScale:
      Optional; A number representing the scaling factor used to reduce
      aggregation tile size; using a larger tileScale (e.g. 2 or 4) may enable
      computations that run out of memory with the default.

  Returns:
    A function that accepts an ee.Image and reduces it by region, according to
    the provided arguments.
  """

  def reduce_regions_function(img):
    """Applies the ee.Image.reduceRegions() method.

    Args:
      img:
        An ee.Image to reduce to a statistic by region.

    Returns:
      An ee.Feature that contains properties representing the image region
      reduction results per band and the image timestamp formatted as
      milliseconds from Unix epoch (included to enable time series plotting).
    """

    stat = img.reduceRegions(
        collection=collection,
        reducer = reducer
        scale=scale,
        crs=crs)

    return ee.Feature(collection, stat).set({'millis': img.date().millis()})
  return reduce_regions_function

In [115]:
# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
    """Extracts the values from a FeatureCollection and stores them as a list in ee.Dictionary. ee.Dictionary
    can be converted to a pandasDataframe if needed. 
    
    Args:
    ee.featureCollection:
      An ee.featureCollection contains images and properties that can be converted 
      to a dictionary with geometries. 
      
    Returns:
    A ee.Dictionary (useful for converting to Pandas objects)
    """
    prop_names = fc.first().propertyNames()
    prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

    return ee.Dictionary.fromLists(prop_names, prop_lists)